Модель предсказывает автор по куску текста

In [19]:
import tensorflow as tf
import pandas as pd
import numpy as np
import collections
from sklearn.cross_validation import train_test_split


data = pd.read_csv('dataset.csv')
data.shape

(19579, 3)

In [20]:
X = data.drop(('author'), axis=1)
y = data['author']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)


In [18]:
sentence = [i for i in X_train['text']]
authors = [i for i in y_train]

info = {sentence[i]:authors[i] for i in range(len(X_train['text']))}


{'His Majesty looked chagrined.': 'EAP',
 'Our conversation was now long, earnest, uninterrupted, and totally unreserved.': 'EAP',
 "Dxn't crxw, anxther time, befxre yxu're xut xf the wxxds Dxes yxur mxther knxw yxu're xut?": 'EAP',
 'This burst of passionate feeling over, with calmed thoughts we sat together, talking of the past and present.': 'MWS',
 'The idea which the raconteur has either failed to entertain clearly, or has sacrificed in its expression to his national love of point, is, doubtless, the very tenable one that the higher order of music is the most thoroughly estimated when we are exclusively alone.': 'EAP',
 'And always the goal of my fancies was the mighty vine grown wall with the little gate of bronze therein.': 'HPL',
 'Thou shalt shew me the ways of travel and I will attend thy songs at evening when the stars one by one bring dreams to the minds of dreamers.': 'HPL',
 'His third, and most difficult of realization, was the contempt of ambition.': 'EAP',
 'He had sca

In [21]:
from gensim.models import Word2Vec
import nltk

text_train = X_train['text'].values.tolist()
author_train = y_train.values.tolist()

corp = text_train + author_train

nltk_tok = [nltk.word_tokenize(sent) for sent in corp]


In [22]:
model = Word2Vec(nltk_tok, min_count=1, size=40, hs=1, negative=0)

In [23]:
# Преобразвание обучающих данных

new_text_vec = []
for i in X_train['text']:
    text = model.score([i])
    new_text_vec.append(text)

In [24]:
model.score(['Our conversation was now long, earnest, uninterrupted, and totally unreserved.'.split()])

array([-43.735218], dtype=float32)

In [77]:
text = []
for i in range(len(new_text_vec)):
    text.append(float(new_text_vec[i].view()[0]))
X_train_new = pd.DataFrame(columns=['id','text'])
X_train_new['text'] = text
X_train_new['id'] = [i for i in range(len(text))]
X_train_new['author'] = y_train
X_train_new = X_train_new.fillna(X_train_new.mean())

In [82]:
X_train_new['author'] = X_train_new['author'].fillna(0)
X_train_new = X_train_new[X_train_new.author != float(0)]


In [90]:
# Преобразование тестовых данных

text_vec = []
for i in X_test['text']:
    text = model.score([i])
    text_vec.append(text)

In [93]:
text_test = []
for i in range(len(text_vec)):
    text_test.append(float(text_vec[i].view()[0]))
X_test_new = pd.DataFrame(columns=['id','text'])
X_test_new['text'] = text_test
X_test_new['id'] = [i for i in range(len(text_test))]
X_test_new['author'] = y_test
X_test_new = X_test_new.fillna(X_test_new.mean())

In [98]:
X_test_new = X_test_new.fillna(0)
X_test_new = X_test_new[X_test_new.author != 0]

In [126]:
X_train = X_train_new.drop(('author'), axis=1)
y_train = X_train_new['author']
X_test = X_test_new.drop(('author'), axis=1)
y_test = X_test_new['author']


1777

In [135]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=45, max_depth = 24, min_samples_leaf=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [137]:
from sklearn.metrics import f1_score, confusion_matrix

f1_score(y_test, y_pred, average='macro')
# confusion_matrix(y_test, y_pred)

0.3180690910388189

In [134]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.accuracy_score(y_test, y_pred)


0.38379290939786154